In [27]:
!which python

/Users/apabook/Desktop/ds-deployment/venv/bin/python


In [28]:
import requests
import json
import pandas as pd
from urllib.error import HTTPError, URLError

# Complete dataspace flow

## Consts


In [29]:
data_space_provider = "http://127.0.0.1:1200"
data_space_consumer = "http://127.0.0.1:1100"
api_endpoint = "https://jsonplaceholder.typicode.com/comments"

data_space_provider_participant_id = ""
data_space_provider_participant_token = ""
data_space_consumer_participant_id = ""
data_space_consumer_participant_token = ""
catalog_id = ""
dataset_id = ""
distribution_id = ""
data_service_id = ""
policy_id = ""
target_id = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""
consumer_data_address = ""

json_header = {
    "Content-Type": "application/json",
}

## Wallets setup

In [30]:
url = data_space_provider + "/api/v1/wallet/onboard"
response = requests.request("POST", url)
response

<Response [201]>

In [31]:
url = data_space_consumer + "/api/v1/wallet/onboard"
response = requests.request("POST", url)
response

<Response [201]>

## Participants onboarding

### Current Participants information

In [32]:
url = data_space_provider + "/api/v1/mates/me"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoiWXU4b3dhN3pHZ191Vi1XN3ZpM0JVMnA1aExPTGNCZDNMQkRNXzY4ZUR4QSIsIngiOiIxSzFWc1lrZ3RmREh3WHJfUEF5X25FeFh1ZmIxT3NLQjloYXB4aEkwcl80In0",
    "participant_slug": "Myself",
    "participant_type": "Provider",
    "base_url": "http://127.0.0.1:1200",
    "token": null,
    "token_actions": null,
    "saved_at": "2025-07-02T13:02:37.890165",
    "last_interaction": "2025-07-03T11:04:54.330953",
    "is_me": true
}


In [33]:
url = data_space_consumer + "/api/v1/mates/me"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoibWlwSXF5TXpYLUpaQ3B6dnFmOFZTWDB5RlhjWjlqWDdMcHI5bDZVUGx0VSIsIngiOiJFX19lMXZIMm9rMDBUSEVTTkFIUXpNNl85b2pmWXJscGo2MzROZVBTdU9VIn0",
    "participant_slug": "Myself",
    "participant_type": "Consumer",
    "base_url": "http://127.0.0.1:1100",
    "token": null,
    "token_actions": null,
    "saved_at": "2025-07-02T13:02:45.937593",
    "last_interaction": "2025-07-03T11:04:58.467518",
    "is_me": true
}


### Consumer registering in Provider via Verifiable presentation

In [34]:
url = data_space_provider + "/api/v1/.well-known/did.json"
response = requests.request("GET", url)
response_as_json = response.json()
data_space_provider_participant_id = response_as_json["id"]
data_space_provider_participant_id

'did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoiWXU4b3dhN3pHZ191Vi1XN3ZpM0JVMnA1aExPTGNCZDNMQkRNXzY4ZUR4QSIsIngiOiIxSzFWc1lrZ3RmREh3WHJfUEF5X25FeFh1ZmIxT3NLQjloYXB4aEkwcl80In0'

In [36]:
url = data_space_consumer + "/api/v1/auth/manual/ssi"
payload = {
    "url": data_space_provider + "/api/v1/access",
    "id": data_space_provider_participant_id,
    "slug": "provider",
    "actions": "talk"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_text = response.text
response_as_text

'openid4vp://authorize?response_type=vp_token&client_id=http%3A%2F%2Fhost.docker.internal%3A1200%2Fapi%2Fv1%2Fverify%2F9ZJpk42eP09x&response_mode=direct_post&presentation_definition_uri=http%3A%2F%2Fhost.docker.internal%3A1200%2Fapi%2Fv1%2Fpd%2F9ZJpk42eP09x&client_id_scheme=redirect_uri&nonce=TVDITXLYihjX&response_uri=http%3A%2F%2Fhost.docker.internal%3A1200%2Fapi%2Fv1%2Fverify%2F9ZJpk42eP09x'

### Provider and Consumer ids

In [40]:
url = data_space_provider + "/api/v1/mates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_consumer_participant_id = response_as_json[1]["participant_id"]
data_space_consumer_participant_token = response_as_json[1]["token"]
print(json.dumps(response_as_json, indent=4))

[
    {
        "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoibWlwSXF5TXpYLUpaQ3B6dnFmOFZTWDB5RlhjWjlqWDdMcHI5bDZVUGx0VSIsIngiOiJFX19lMXZIMm9rMDBUSEVTTkFIUXpNNl85b2pmWXJscGo2MzROZVBTdU9VIn0",
        "participant_slug": "\"rainbow_consumer\"",
        "participant_type": "Consumer",
        "base_url": "http://127.0.0.1:1100",
        "token": "EwxZU3uCVy0KpqEHX0oxXE25uMRiwFA4UNdT4ID6Nhs",
        "token_actions": "talk",
        "saved_at": "2025-07-02T13:03:17.663910",
        "last_interaction": "2025-07-02T13:03:17.663911",
        "is_me": false
    },
    {
        "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoiWXU4b3dhN3pHZ191Vi1XN3ZpM0JVMnA1aExPTGNCZDNMQkRNXzY4ZUR4QSIsIngiOiIxSzFWc1lrZ3RmREh3WHJfUEF5X25FeFh1ZmIxT3NLQjloYXB4aEkwcl80In0",
        "participant_slug": "Myself",
        "participant_type": "Provider",
        "base_url": "http://127.0.0.1:1200",
        "token": null,
        "token_actions": null,
        "sa

In [41]:
url = data_space_consumer + "/api/v1/mates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_provider_participant_id = response_as_json[1]["participant_id"]
data_space_provider_participant_token = response_as_json[1]["token"]
print(json.dumps(response_as_json, indent=4))

[
    {
        "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoiWXU4b3dhN3pHZ191Vi1XN3ZpM0JVMnA1aExPTGNCZDNMQkRNXzY4ZUR4QSIsIngiOiIxSzFWc1lrZ3RmREh3WHJfUEF5X25FeFh1ZmIxT3NLQjloYXB4aEkwcl80In0",
        "participant_slug": "provider",
        "participant_type": "Provider",
        "base_url": "http://127.0.0.1:1200",
        "token": "EwxZU3uCVy0KpqEHX0oxXE25uMRiwFA4UNdT4ID6Nhs",
        "token_actions": "talk",
        "saved_at": "2025-07-02T13:03:17.685699",
        "last_interaction": "2025-07-02T13:03:17.685700",
        "is_me": false
    },
    {
        "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoibWlwSXF5TXpYLUpaQ3B6dnFmOFZTWDB5RlhjWjlqWDdMcHI5bDZVUGx0VSIsIngiOiJFX19lMXZIMm9rMDBUSEVTTkFIUXpNNl85b2pmWXJscGo2MzROZVBTdU9VIn0",
        "participant_slug": "Myself",
        "participant_type": "Consumer",
        "base_url": "http://127.0.0.1:1100",
        "token": null,
        "token_actions": null,
        "saved_at": "20

## Create Catalog instances and metadata

In [42]:
url = data_space_provider + "/api/v1/catalogs"
payload = {
    "foaf:homepage": "My dataspace catalog",
    "dct:title": "My dataspace catalog",

}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]
catalog_id

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [43]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets".format(catalog_id)
payload = {
    "dct:title": "My dataspace dataset",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["@id"]
dataset_id

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [44]:
url = data_space_provider + "/api/v1/catalogs/{}/data-services".format(catalog_id)
payload = {
    "dcat:endpointURL": api_endpoint
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_service_id = response_as_json["@id"]
data_service_id

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [45]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets/{}/distributions".format(catalog_id, dataset_id)
payload = {
    "dcat:accessService": data_service_id,
    "dct:formats": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
distribution_id = response_as_json["@id"]
distribution_id

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Create policies

In [46]:
url = data_space_provider + "/api/v1/datasets/{}/policies".format(dataset_id)
payload = {
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }

    ],
    "obligation": [],
    "prohibition": []
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["@id"]
target_id = response_as_json["target"]
print(json.dumps(response_as_json, indent=4))

KeyError: '@id'

## Contract Negotiation

In [47]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

KeyError: 'consumerPid'

In [ ]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id, # remove to test offer from provider
    "providerPid": cn_provider_id, # remove to test offer from provider
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

In [ ]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

In [ ]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

In [ ]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-acceptance"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

In [ ]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-agreement"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
agreement_id=response_as_json["agreement"]["@id"]
print(json.dumps(response_as_json, indent=2))

In [ ]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-verification"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

In [ ]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-finalization"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

## Transfer Negotiation


In [ ]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "agreementId": agreement_id,
    "format": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_consumer_id = response_as_json["consumerPid"]
transfer_provider_id = response_as_json["providerPid"]
callback_address = response_as_json["callbackAddress"]
print(json.dumps(response_as_json, indent=2))

In [ ]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-start"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

In [ ]:
url = data_space_consumer + "/api/v1/dataplane/" + transfer_consumer_id
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
consumer_data_address = response_as_json["process_address"]["url"]
print(json.dumps(response_as_json, indent=2))

In [ ]:
try :
    df = pd.read_json(consumer_data_address)
    print(df)
except HTTPError as e:
    print(e)


In [ ]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

In [ ]:
try :
    df = pd.read_json(consumer_data_address)
    print(df)
except HTTPError as e:
    print(e)

In [ ]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-start"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

In [ ]:
try :
    df = pd.read_json(consumer_data_address)
    print(df)
except HTTPError as e:
    print(e)

In [ ]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

In [ ]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-completion"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))